# PostgreSQL and AutoCommits.



In [1]:
import psycopg2

Connect to local database

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=janfrodeno")

Get a cursor, which is used to execute queries.

In [3]:
cur = conn.cursor()

Lets try to run a query.

In [4]:
cur.execute("select * from test123")

UndefinedTable: relation "test123" does not exist
LINE 1: select * from test123
                      ^


Since the previous query failed, we will create the desired table to now make it work

In [5]:
cur.execute("CREATE TABLE test123 (col1 int, col2 int, col3 int)")

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


The previous query produced a strange error. Since PostgreSQL is ACID, each transaction needs to be commited in order to ensure validity of data. Since our firs query failed, that transaction was left unncommited. Whenever a transaction is uncommited, PostgreSQL will block until commit or restart.

To solve this problem we use autocommit. Lets restart the connection

In [13]:
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=janfrodeno")

In [14]:
conn.set_session(autocommit=True)
cur = conn.cursor()

We replicate the error:

In [15]:
cur.execute("select * from test123")

UndefinedTable: relation "test123" does not exist
LINE 1: select * from test123
                      ^


Although the previous line failed because the table does not exist, the transactio nhas been commited and so executed is not blocked. Hence, we can continue to create the table because in any case we have autocommited.

In [16]:
cur.execute("CREATE TABLE test123 (col1 int, col2 int, col3 int)")

The table exists now (but is empty)

In [17]:
cur.execute("select * from test123")

In [18]:
cur.execute("select count(*) from test123")
print(cur.fetchall())

[(0,)]


In [19]:
cur.execute("drop table test123")